In [13]:
import pandas as pd
import numpy as np

In [ ]:
snli_dataset = pd.read_csv('testing/SNLI/snli_1.0_test.csv')

snli_dataset.drop(columns=['sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'captionID', 'pairID', 'label1', 'label2','label3', 'label4', 'label5'], inplace=True)

snli_dataset['gold_label'] = snli_dataset['gold_label'].map({'neutral': 0, 'entailment': 1, 'contradiction': -1})

print(snli_dataset.__len__())

10000


In [ ]:
from classifier import classify_text_binary

testing_num = 10000  # Number of rows to test

num_neutral = 0
num_agree = 0
num_disagree = 0

num_correct = 0
num_incorrect = 0
num_correct_ignore_neutral_diff = 0
num_incorrect_ignore_neutral = 0

num_actual_neutral = np.sum(snli_dataset['gold_label'][:testing_num] == 0)
num_actual_agree = np.sum(snli_dataset['gold_label'][:testing_num] == 1)
num_actual_disagree = np.sum(snli_dataset['gold_label'][:testing_num] == -1)


for i in range(testing_num):
    body = snli_dataset['sentence1'][i]
    headline = snli_dataset['sentence2'][i]
    results = classify_text_binary(headline, body, score_threshold=0.55)
    stance = results[1]

    print(f"Sentence 1: {headline}")
    print(f"Sentence 2: {body}")

    if stance == 0:
        num_neutral += 1
    elif stance == 1:
        num_agree += 1
    elif stance == -1:
        num_disagree += 1

    if stance == snli_dataset['gold_label'][i]:
        num_correct += 1
        print(f"Classification result: {stance}")
        print(f"Dataset {i}: The classification is correct.")
    else:
        if (stance == 0 and snli_dataset['gold_label'][i] != 0):
            num_incorrect_ignore_neutral += 1
        num_incorrect += 1
        print(f"Dataset {i}: The classification is incorrect.")
        print(f"Expected: {snli_dataset['gold_label'][i]}, Got: {stance}")
    print("—"*10)

{'entailing': [0.39644497632980347, 0.4936041533946991], 'contradicting': [0.6035550236701965, 0.5063958168029785]}
Sentence 1: The church has cracks in the ceiling.
Sentence 2: This church choir sings to the masses as they sing joyous songs from the book at a church.
Dataset 0: The classification is incorrect.
Expected: 0.0, Got: -1
——————————
{'entailing': [0.5904393792152405, 0.7339743375778198], 'contradicting': [0.4095606207847595, 0.26602569222450256]}
Sentence 1: The church is filled with song.
Sentence 2: This church choir sings to the masses as they sing joyous songs from the book at a church.
Classification result: 1
Dataset 1: The classification is correct.
——————————
{'entailing': [0.1923847794532776, 0.38613471388816833], 'contradicting': [0.8076152205467224, 0.6138652563095093]}
Sentence 1: A choir singing at a baseball game.
Sentence 2: This church choir sings to the masses as they sing joyous songs from the book at a church.
Classification result: -1
Dataset 2: The clas

In [ ]:
print(f"Number of (agree/neutral/disagree) stances in the dataset: {num_actual_agree}/{num_actual_neutral}/{num_actual_disagree}")
print(f"Number of (agree/neutral/disagree) classifications: {num_agree}/{num_neutral}/{num_disagree}")
print(f"Accuracy: {num_correct / (num_correct + num_incorrect) * 100:.2f}%")
print(f"Accuracy ignoring neutral: {(num_correct + num_incorrect_ignore_neutral) / (num_incorrect+num_correct) * 100:.2f}%")

Number of (agree/neutral/disagree) stances in the dataset: 3368/3219/3237
Number of (agree/neutral/disagree) classifications: 3415/3120/3465
Accuracy: 62.30%
Accuracy ignoring neutral: 78.36%
